In [ ]:
import numpy as np
import scipy.linalg as spla
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

from pymor.algorithms.svd_va import mos, qr_svd
from pymor.vectorarrays.numpy import NumpyVectorSpace

# Setup

In [ ]:
n = 1000
m = 50
random = np.random.RandomState(0)
U0 = spla.qr(random.standard_normal((n, m)), mode='economic')[0]
s0 = np.logspace(-20, 1, m)[::-1]
Vh0 = spla.qr(random.standard_normal((m, m)), mode='economic')[0]
A = U0 * s0 @ Vh0.T
Ava = NumpyVectorSpace(n).from_numpy(A.T)

In [ ]:
U, s, Vh = spla.svd(A, lapack_driver='gesvd')

In [ ]:
fig, ax = plt.subplots()
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.semilogy(s0, '.-', label='Exact')
ax.semilogy(s, '.-', label='GESVD')
ax.set_title('Singular values')
ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.semilogy(np.abs(s - s0) / s0, '.-')
ax.set_title('Relative error of GESVD singular values')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.semilogy([spla.norm(U[:, :r] - U0[:, :r] @ U0[:, :r].T @ U[:, :r], ord=2)
             for r in range(1, m + 1)], '.-')
ax.set_title('Errors in dominant left subspaces')
plt.show()

# Method of snapshots

In [ ]:
U_mos, s_mos, Vh_mos = mos(Ava, rtol=0)
U_mos = U_mos.to_numpy().T

In [ ]:
len(s_mos)

In [ ]:
fig, ax = plt.subplots()
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.semilogy(s0, '.-', label='Exact')
ax.semilogy(s, '.-', label='GESVD')
ax.semilogy(s_mos, '.-', label='mos')
ax.set_title('Singular values')
ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.semilogy(np.abs(s_mos - s0[:len(s_mos)]) / s0[:len(s_mos)], '.-')
ax.set_title('Relative distance of mos and exact singular values')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.semilogy(np.abs(s_mos - s[:len(s_mos)]) / s[:len(s_mos)], '.-')
ax.set_title('Relative distance of mos and GESVD singular values')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.semilogy([spla.norm(U[:, :r] - U0[:, :r] @ U0[:, :r].T @ U[:, :r], ord=2)
             for r in range(1, m + 1)], '.-', label='GESVD - Exact')
ax.semilogy([spla.norm(U_mos[:, :r] - U0[:, :r] @ U0[:, :r].T @ U_mos[:, :r], ord=2)
             for r in range(1, len(s_mos))], '.-', label='mos - Exact')
ax.semilogy([spla.norm(U_mos[:, :r] - U[:, :r] @ U[:, :r].T @ U_mos[:, :r], ord=2)
             for r in range(1, len(s_mos))], '.--', label='mos - GESVD')
ax.set_title('Errors in dominant left subspaces')
ax.legend()
plt.show()

# QR + SVD

In [ ]:
U_qr, s_qr, Vh_qr = qr_svd(Ava, rtol=0)
U_qr = U_qr.to_numpy().T

In [ ]:
fig, ax = plt.subplots()
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.semilogy(s0, '.-', label='Exact')
ax.semilogy(s, '.-', label='GESVD')
ax.semilogy(s_qr, '.-', label='Gram-Schmidt + SVD')
ax.set_title('Singular values')
ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.semilogy(np.abs(s_qr - s0) / s0, '.-')
ax.set_title('Relative distance of Gram-Schmidt+SVD and exact singular values')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.semilogy(np.abs(s_qr - s) / s, '.-')
ax.set_title('Relative distance of Gram-Schmidt+SVD and GESVD singular values')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.semilogy([spla.norm(U[:, :r] - U0[:, :r] @ U0[:, :r].T @ U[:, :r], ord=2)
             for r in range(1, m + 1)], '.-', label='GESVD - Exact')
ax.semilogy([spla.norm(U_qr[:, :r] - U0[:, :r] @ U0[:, :r].T @ U_qr[:, :r], ord=2)
             for r in range(1, len(s_mos))], '.--', label='QR - Exact')
ax.semilogy([spla.norm(U_qr[:, :r] - U[:, :r] @ U[:, :r].T @ U_qr[:, :r], ord=2)
             for r in range(1, len(s_mos))], '.--', label='QR - GESVD')
ax.set_title('Errors in dominant left subspaces')
ax.legend()
plt.show()